In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import json
import os
import re
import warnings
import logging

In [ ]:
logging.basicConfig(level=logging.INFO)

In [ ]:
import cv2

import matplotlib.pyplot as plt
from matplotlib.lines import Line2D
from matplotlib.collections import PatchCollection
from matplotlib.patches import Rectangle
plt.rcParams["font.family"] = "DejaVu Serif"

plt.style.use("tableau-colorblind10")  # [#006BA4, #FF800E, #ABABAB, #595959, #5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF]
plt.rcParams['axes.axisbelow'] = True

import numpy as np
from numba import njit, prange
import pandas as pd
from PIL import Image
import rasterio
import xarray as xr

In [ ]:
# Load pandora imports
#import pandora
%aimport pandora
#pandora.setup_logging(True)

In [ ]:
# Load plugins
pandora.import_plugin()

### Defining functions for loading Middlebury GT and for correctly plotting greyscale images.

In [ ]:
def process_pfm(disp_path):
    """Read Middlebury disparity map and calculate depth map.
    http://davis.lbl.gov/Manuals/NETPBM/doc/pfm.html
    """
    disp = cv2.imread(disp_path, cv2.IMREAD_UNCHANGED)
    with open(disp_path, "rb") as pfm_file:
        header = pfm_file.readline().decode().rstrip()
        channels = 3 if header == "PF" else 1

        dim_match = re.match(r"^(\d+)\s(\d+)\s$", pfm_file.readline().decode("utf-8"))
        if dim_match:
            width, height = map(int, dim_match.groups())
        else:
            raise Exception("Malformed PFM header.")

        scale = float(
            pfm_file.readline().decode().rstrip()
        )  # read disparity scale factor
        if scale < 0:
            endian = "<"  # littel endian
            scale = -scale
        else:
            endian = ">"  # big endian

    disp = disp * scale
    return disp

def normalize_for_imshow(img: np.ndarray):
    img_normalized = np.zeros(img.shape, dtype=img.dtype)
    for band in range(img.shape[-1]):
        min_band, max_band = np.nanquantile(img[:,:,band], [0.01, 0.99])
        img_normalized[:,:,band] = np.clip(img[:,:,band], min_band, max_band)
        img_normalized[:,:,band] = (img_normalized[:,:,band] - np.nanmin(img_normalized[:,:,band])) / (np.nanmax(img_normalized[:,:,band]) - np.nanmin(img_normalized[:,:,band]))
    return img_normalized

def crop_img(path):
    img = np.array(Image.open(path))
    img_sum = img.sum(axis=2)
    for k in range(img.shape[0]):
        if (img_sum[k, :] != (255*4)).any():
            top = k
            break
    for k in range(img.shape[0]):
        if (img_sum[img.shape[0] - 1 - k, :] != (255*4)).any():
            bottom = img.shape[0] - 1 - k
            break
    for k in range(img.shape[1]):
        if (img_sum[:, k] != (255*4)).any():
            left = k
            break
    for k in range(img.shape[1]):
        if (img_sum[:, img.shape[1] - 1 - k] != (255*4)).any():
            right = img.shape[1] - 1 - k
            break
    img = img[top:bottom+1, left:right+1]
    Image.fromarray(img).save(path)

# Investigation

### Defining the scene, Cost function used, input and output paths.
* For Middlebury data, `year` should be in [2003, 2005, 2006, 2014, 2021]
* For Jacksonville or Montepellier data, `year` is "".

`scene` is the name of the scene. For Middlebury it is "cones" for example. For Montpellier or Jacksonville it is something like "MTP_144".
Check `/work/CAMPUS/users/malinoro/Data/Satellite` and `/work/CAMPUS/users/malinoro/Data/MiddleBury` to see which scenes are available

In [ ]:
cost_function = "CENSUS"
year, scene = "2003", "cones"


if cost_function == "CENSUS":
    cost_func_pandora = "census"
    penalty_method = "sgm_penalty"
    win_size = 5
elif cost_function == "MCCNN":
    cost_func_pandora = "mc_cnn"
    penalty_method = "mc_cnn_fast_penalty"
    win_size = 11
else:
    logging.error("Cost function is not supported, add sgm_penalty, win_size etc on your own!")
    raise NameError


if year == "":
    root_gt = "/work/CAMPUS/users/malinoro/Data/Satellite"
else:
    root_gt = "/work/CAMPUS/users/malinoro/Data/MiddleBury"

output_path = os.path.join("/work/CAMPUS/users/malinoro/outputs/Pandora", cost_function, scene)
gt_path = os.path.join(root_gt, year, scene)
assert os.path.exists(gt_path)

### Loading images, GT, d_min, d_max on disk

In [ ]:
if year in ["2003", "2005", "2006"]:
    img_left_path = os.path.join(gt_path, "im2_gray.png")
    img_right_path = os.path.join(gt_path, "im6_gray.png")
    
    img_left = np.array(Image.open(os.path.join(gt_path, "im2.png")))
    img_left_gray = np.array(Image.open(img_left_path))
    img_right = np.array(Image.open(os.path.join(gt_path, "im6.png")))
    img_right_gray = np.array(Image.open(img_right_path))
    
    scale_gt = 4 if year=="2003" else 3
    true_disp = - np.asarray(Image.open(os.path.join(gt_path, "disp2.png"))).astype(float) / scale_gt
    true_disp[true_disp == 0] = np.nan

    d_min, d_max = -60, 0
    
elif year in ["2014", "2021"]:
    img_left_path = os.path.join(gt_path, "im0_gray.png")
    img_right_path = os.path.join(gt_path, "im1_gray.png")
    
    img_left = np.array(Image.open(os.path.join(gt_path, "im0.png")))
    img_left_gray = np.array(Image.open(img_left_path))
    img_right = np.array(Image.open(os.path.join(gt_path, "im1.png")))
    img_right_gray = np.array(Image.open(img_right_path))
    
    true_disp = -process_pfm(os.path.join(gt_path, "disp0.pfm"))
    true_disp[true_disp == -np.inf] = np.nan
    
    with open(os.path.join(gt_path, "calib.txt"), "r") as f:
        lines = f.readlines()
    d_min , d_max = - int(lines[6].split("=")[-1]), 0
    
elif year in ["2001"]:
    img_left_path = os.path.join(gt_path, "im2_gray.png")
    img_right_path = os.path.join(gt_path, "im6_gray.png")
    
    img_left = np.array(Image.open(os.path.join(gt_path, "im2.png")))
    img_left_gray = np.array(Image.open(img_left_path))
    img_right = np.array(Image.open(os.path.join(gt_path, "im6.png")))
    img_right_gray = np.array(Image.open(img_right_path))
    
    scale_gt = 8
    true_disp = - cv2.imread(os.path.join(gt_path, "disp2.pgm"))[:,:,0].astype(float) / scale_gt
    tmp_disp = np.unique(true_disp)
    d_min, d_max = np.floor(np.min(tmp_disp)).astype(int), np.ceil(np.max(tmp_disp)).astype(int)
    true_disp[true_disp == 0] = np.nan


elif "MTP_" in scene:
    img_left_path = os.path.join(gt_path, "left_epipolar_image_gray.tif")
    img_right_path = os.path.join(gt_path, "right_epipolar_image_gray.tif")
    
    img_left = np.moveaxis(rasterio.open(os.path.join(gt_path, "left_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_left = normalize_for_imshow(img_left)
    img_left_gray = rasterio.open(img_left_path).read(1)
    img_right = np.moveaxis(rasterio.open(os.path.join(gt_path, "right_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_right = normalize_for_imshow(img_right)
    img_right_gray = rasterio.open(img_right_path).read(1)

    true_disp = rasterio.open(os.path.join(gt_path, "left_epipolar_disp.tif")).read(1)
    true_disp_mask = rasterio.open(os.path.join(gt_path, "left_epipolar_mask.tif")).read(1)
    true_disp[true_disp_mask > 0] = np.nan
    
    d_min, d_max = int(np.floor(np.nanmin(true_disp))), int(np.ceil(np.nanmax(true_disp)))
    
    
elif "JAX_" in scene:
    img_left_path = os.path.join(gt_path, "left_epipolar_image_gray.tif")
    img_right_path = os.path.join(gt_path, "right_epipolar_image_gray.tif")
    
    img_left = np.moveaxis(rasterio.open(os.path.join(gt_path, "left_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_left_gray = rasterio.open(img_left_path).read(1)
    img_right = np.moveaxis(rasterio.open(os.path.join(gt_path, "right_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_right_gray = rasterio.open(img_right_path).read(1)

    true_disp = rasterio.open(os.path.join(gt_path, "left_epipolar_disp.tif")).read(1)
    true_disp_mask = rasterio.open(os.path.join(gt_path, "left_epipolar_mask.tif")).read(1)
    true_disp[true_disp_mask > 0] = np.nan
    
    d_min, d_max = int(np.floor(np.nanmin(true_disp))), int(np.ceil(np.nanmax(true_disp)))

logging.info(f" {scene}\nd_min: {d_min} | d_max: {d_max}\nwin_size: {win_size}")

#### Computing with Pandora

In [ ]:
input_config = {
    "left": {"img": img_left_path, "mask": None, "disp": [-60, 0], "nodata": np.nan},
    "right": {"img": img_right_path, "mask": None, "disp": None, "nodata": np.nan},
}

img_left_pandora = pandora.img_tools.create_dataset_from_inputs(input_config=input_config["left"])
img_right_pandora = pandora.img_tools.create_dataset_from_inputs(input_config=input_config["right"])

pandora.check_configuration.check_datasets(img_left_pandora, img_right_pandora)

pandora_machine = pandora.state_machine.PandoraMachine()

In [ ]:
user_cfg = {
    "input": {
        "left": {"img": img_left_path,
                 "disp": [d_min, d_max]},
        "right": {"img": img_right_path}
    },
    "pipeline": {
        "matching_cost": {
            "matching_cost_method": cost_func_pandora,
            "window_size": win_size,
            "subpix": 1
        },
        "optimization": {
            "optimization_method": "sgm",
            "overcounting": False,
            "penalty": {
                "penalty_method": penalty_method,
            },
            "sgm_version": "c++",
            "min_cost_paths": False,
            "use_confidence": False
        },
        "cost_volume_confidence": {
            "confidence_method": "ambiguity",
                "eta_max": 0.99,
                "eta_step": 0.04
        },
        "cost_volume_confidence.int": {
            "confidence_method": "interval_bounds",
        },
        "disparity": {"disparity_method": "wta", "invalid_disparity": "NaN"},
        "refinement": {"refinement_method": "vfit"},
        "filter": {"filter_method": "median", "filter_size": 3},
        "filter.int": {"filter_method": "median_for_intervals",
                       "filter_size": 3,
                       "regularization": True,
                       "interval_indicator": "int",
                       "vertical_depth": 2,
                       "quantile_regularization": 0.90
                      },
        "validation": {
            "validation_method": "cross_checking_accurate",
            "cross_checking_threshold": 1.0
        },
    }
}

In [ ]:
metadata_left = pandora.img_tools.get_metadata(user_cfg["input"]["left"]["img"], (d_min, d_max))
metadata_right = pandora.img_tools.get_metadata(user_cfg["input"]["right"]["img"])
user_cfg_pipeline = pandora.check_configuration.get_config_pipeline(user_cfg)

pandora_machine.run_prepare(user_cfg, img_left_pandora, img_right_pandora)

In [ ]:
pandora_machine.run('matching_cost', user_cfg)
pandora_machine.run('optimization', user_cfg)
pandora_machine.run('cost_volume_confidence', user_cfg)
pandora_machine.run('cost_volume_confidence.int', user_cfg)
conf_inf_no_reg = pandora_machine.left_disparity.confidence_measure.sel({"indicator": "confidence_from_interval_bounds_inf.int"}).data.copy()
conf_sup_no_reg = pandora_machine.left_disparity.confidence_measure.sel({"indicator": "confidence_from_interval_bounds_sup.int"}).data.copy()
pandora_machine.run('disparity', user_cfg)
pandora_machine.run('refinement', user_cfg)
pandora_machine.run('filter', user_cfg)
pandora_machine.run('filter.int', user_cfg)
pandora_machine.run('validation', user_cfg)

In [ ]:
pandora_machine.left_disparity

In [ ]:
conf_inf = pandora_machine.left_disparity.confidence_measure.sel({"indicator": "confidence_from_interval_bounds_inf.int"}).data
conf_sup = pandora_machine.left_disparity.confidence_measure.sel({"indicator": "confidence_from_interval_bounds_sup.int"}).data

conf_amb = pandora_machine.left_cv.confidence_measure.sel({"indicator": "confidence_from_ambiguity"}).data

disp_map = pandora_machine.left_disparity.disparity_map.data
validity_map = pandora_machine.left_disparity["validity_mask"].data

logging.info("Pandora finished!")

# Getting bitmap out of the validity_map
validity_bitmap = np.zeros(shape=validity_map.shape + (3,), dtype=bool)
validity_bitmap[:, :, 0] = validity_map & (1<<2)
validity_bitmap[:, :, 1] = validity_map & (1<<8)
validity_bitmap[:, :, 2] = validity_map & (1<<11)

PANDORA_MSK_PIXEL_INVALID  = (validity_map & 0b01111000011) > 0

#no_data = np.isnan(conf_inf_90) | np.isnan(conf_sup_90) | np.isnan(true_disp) # | (validity_bitmap[:,:,8] > 0)
no_data = np.isnan(true_disp)
logging.info("Done")

In [ ]:
wrong_interval = (true_disp < conf_inf) | (true_disp > conf_sup)

logging.info("Percentage of wrong intervals: %s" % (np.nansum(wrong_interval[~no_data])/wrong_interval[~no_data].size*100))

## Visualization of the intervals

In [ ]:
row = 110

### Stats for thetile
error_tot = np.nansum(wrong_interval[~no_data])/wrong_interval[~no_data].size*100
error_row = np.nansum(wrong_interval[row, :][~no_data[row, :]])/wrong_interval[row, :][~no_data[row, :]].size*100

### For the gray rectangles
pandora_invalid = PANDORA_MSK_PIXEL_INVALID[row, :]

### Creating the top left image 
image_error = np.full((*wrong_interval.shape, 4), 0, dtype=np.uint8)
image_error[wrong_interval, :] = (0, 0, 255, 255)
image_error[no_data, :] = (0, 255, 0, 255)

fig, axes = plt.subplots(2, 1, figsize=(20, 20))
axes[0].set_visible(False)

# Ax
ax = fig.add_subplot(221)

ax.imshow(img_left_gray, vmin=np.nanquantile(img_left_gray, 0.01), vmax=np.nanquantile(img_left_gray, 0.99), cmap="gray")
ax.imshow(image_error)
ax.set_title("Wrong interval (reg after)")
legend_elements = [
    Line2D([0], [0], marker="s", color="w", label="Wrong interval", markerfacecolor="b", markersize=15, markeredgecolor="k", markeredgewidth=1),
    Line2D([0], [0], marker="s", color="w", label="No data GT", markerfacecolor="g", markersize=15, markeredgecolor="k", markeredgewidth=1),
    ]
#Line2D([0], [0], marker="s", color="w", label="Validity mask", markerfacecolor="pink", markersize=15, markeredgecolor="k", markeredgewidth=1)
ax.legend(handles=legend_elements, loc="upper left", frameon=True, framealpha=1, facecolor="white")
ax.grid(color="gray", linewidth=1)
ax.axhline(row - win_size//2, c="r")
ax.axhline(row + win_size//2, c="r")

ax.grid(False)

### Left colored image plotting
ax = fig.add_subplot(222)
ax.imshow(img_left)
ax.grid(False) 


### Plotting the predicted disparity, gt, intervals, for the selected row
axes[1].plot(disp_map[row, :], color="b", label="Pandora", linewidth=1)

axes[1].plot(conf_inf[row, :], linestyle="--", color="b", label="Interval reg", linewidth=1)
axes[1].plot(conf_sup[row, :], linestyle="--", color="b", linewidth=1)

axes[1].plot(conf_inf_no_reg[row, :], linestyle="--", color="r", label="Interval", linewidth=1)
axes[1].plot(conf_sup_no_reg[row, :], linestyle="--", color="r", linewidth=1)


axes[1].plot(true_disp[row, :], "g", label="True disp")

axes[1].set_ylabel("Disparity", color="b")
axes[1].tick_params(axis='y', labelcolor="b")
axes[1].set_xlabel("Columns")
axes[1].set_title(f"{cost_function} - {scene} row {row} | Total error {error_tot:.1f}% ~ Row error {error_row:.1f}%")

axes[1].legend(frameon=True, framealpha=1, facecolor="white")
axes[1].set_ylim([d_min, d_max]);

# Grey area for invalid pixels
bottom_, top_ = axes[1].get_ylim()
diffs = np.diff(np.hstack([[False], pandora_invalid, [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten()

invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
axes[1].add_collection(pc)

axes[1].grid(False)

axes[1].legend(loc='upper right', frameon=True, framealpha=1, facecolor="white");
#plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/%s.png" % scene, dpi=150, bbox_inches="tight");

# TMP

In [ ]:
root_output = "/work/CAMPUS/users/malinoro/outputs/CVPR_tests/"
cost_functions = ["CENSUS", "MCCNN"]
masks = ["None", "Incomplete disp", "Occlusions", "Confident_incomplete", "Not_confident_incomplete"]
methods = ["no_reg", "reg_90", "perfect"]
sub_cols = ["Error", "median", "Q1", "Q3", "D1", "D9", "mean", "N pixel"]

ind_year = []
ind_scenes = []

root_gt = "/work/CAMPUS/users/malinoro/Data/MiddleBury"

for year in ["2001", "2003", "2005", "2006", "2014", "2021"]:
    for scene in [s for s in os.listdir(os.path.join(root_gt, year)) if os.path.isdir(os.path.join(root_gt, year, s)) and s[0]!="."]:
        ind_year += [year]
        ind_scenes += [scene]

inds = [ind_year, ind_scenes]

df = pd.DataFrame()
index=pd.MultiIndex.from_tuples(list(zip(*inds)), names=["year", "scene"])

for year, scene in index:
    df = pd.concat([df, pd.read_csv(os.path.join(root_output, "dataframes", f"{year}_{scene}.csv"), header=[0,1,2,3], index_col=[0,1])])
    
new_index = []

for k in range(len(df.index)):
    new_index += [(df.index[k][0], df.index[k][1].split("-perfect")[0])]
df.index = pd.MultiIndex.from_tuples(new_index, names=["year", "scene"])

df = df.dropna()
df.sort_index(inplace=True)
df.to_csv(os.path.join(root_output, "df_maximized.csv"))
logging.info("Done!")

In [ ]:
df = pd.read_csv("/home/mp/malinoro/CAMPUS/outputs/CVPR_tests/df_maximized.csv", header=[0,1,2,3], index_col=[0,1])
#df = df.sort_values(by=[("CENSUS", "Incomplete disp", "reg_90", "Error")])
df.sort_index(level="year", inplace=True)

In [ ]:
fig = plt.figure(figsize=(15,3))
plt.rcParams["font.family"] = "DejaVu Serif"

ax = fig.add_subplot(111)
ax.axhline(90, linewidth=1, linestyle=":", color="k")


ax.plot(df.index.get_level_values("scene"), df[("MCCNN", "Incomplete disp", "reg_90", "Error")]*100, marker="o", markersize=4, linestyle="-", linewidth=1, label="MCCNN")
ax.plot(df.index.get_level_values("scene"), df[("CENSUS", "Incomplete disp", "reg_90", "Error")]*100, marker="^", markersize=4, linestyle="-", linewidth=1, label="CENSUS")


ax.legend(loc='lower left', frameon=True, framealpha=1, facecolor="white")
ax.set_ylim([75,100.2])
ax.set_xticks(ax.get_xticks(), labels=[f"{y} {s}" for y, s in df.index], rotation=-90) # ha=left for rotating from the left side (i.e. horizontal alignment)
ax.set_xlim([-0.2,82.2])

score_census = (df[("CENSUS", "Incomplete disp", "reg_90", "Error")]*df[("CENSUS", "Incomplete disp", "reg_90", "N pixel")]).sum() / df[("CENSUS", "Incomplete disp", "reg_90", "N pixel")].sum()
score_mccnn = (df[("MCCNN", "Incomplete disp", "reg_90", "Error")]*df[("MCCNN", "Incomplete disp", "reg_90", "N pixel")]).sum() / df[("MCCNN", "Incomplete disp", "reg_90", "N pixel")].sum()

fig.suptitle(f'Percentage of correct intervals');
plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/Perfomances.png", dpi=300, bbox_inches="tight");

In [ ]:
year, scene = "2003", "teddy"
cost_function = "CENSUS"


root_output = "/work/CAMPUS/users/malinoro/outputs/CVPR_tests/"

if year!="":
    root_gt = "/work/CAMPUS/users/malinoro/Data/MiddleBury"
    output_path = os.path.join(root_output, cost_function, scene)
else:
    root_gt = "/work/CAMPUS/users/malinoro/Data/Satellite"
    output_path = os.path.join(root_output, "MTP", cost_function, scene)


gt_path = os.path.join(root_gt, year, scene)

if year in ["2003", "2005", "2006"]:
    img_left_path = os.path.join(gt_path, "im2_gray.png")
    img_right_path = os.path.join(gt_path, "im6_gray.png")
    
    img_left = np.array(Image.open(os.path.join(gt_path, "im2.png")))
    img_left_gray = np.array(Image.open(img_left_path))
    img_right = np.array(Image.open(os.path.join(gt_path, "im6.png")))
    img_right_gray = np.array(Image.open(img_right_path))
    
    scale_gt = 4 if year=="2003" else 3
    true_disp = - np.asarray(Image.open(os.path.join(gt_path, "disp2.png"))).astype(float) / scale_gt
    true_disp[true_disp == 0] = np.nan

    d_min, d_max = -60, 0
    
elif year in ["2014", "2021"]:
    img_left_path = os.path.join(gt_path, "im0_gray.png")
    img_right_path = os.path.join(gt_path, "im1_gray.png")
    
    img_left = np.array(Image.open(os.path.join(gt_path, "im0.png")))
    img_left_gray = np.array(Image.open(img_left_path))
    img_right = np.array(Image.open(os.path.join(gt_path, "im1.png")))
    img_right_gray = np.array(Image.open(img_right_path))
    
    true_disp = -process_pfm(os.path.join(gt_path, "disp0.pfm"))
    true_disp[true_disp == -np.inf] = np.nan
    
    with open(os.path.join(gt_path, "calib.txt"), "r") as f:
        lines = f.readlines()
    d_min , d_max = - int(lines[6].split("=")[-1]), 0
    
    
elif "MTP_" in scene:
    img_left_path = os.path.join(gt_path, "left_epipolar_image_gray.tif")
    img_right_path = os.path.join(gt_path, "right_epipolar_image_gray.tif")
    
    img_left = np.moveaxis(rasterio.open(os.path.join(gt_path, "left_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_left = normalize_for_imshow(img_left)
    img_left_gray = rasterio.open(img_left_path).read(1)
    img_right = np.moveaxis(rasterio.open(os.path.join(gt_path, "right_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_right = normalize_for_imshow(img_right)
    img_right_gray = rasterio.open(img_right_path).read(1)

    true_disp = rasterio.open(os.path.join(gt_path, "left_epipolar_disp.tif")).read(1)
    true_disp_mask = rasterio.open(os.path.join(gt_path, "left_epipolar_mask.tif")).read(1)
    true_disp[true_disp_mask > 0] = np.nan
    
    d_min, d_max = int(np.floor(np.nanmin(true_disp))), int(np.ceil(np.nanmax(true_disp)))
    
    
elif "JAX_" in scene:
    img_left_path = os.path.join(gt_path, "left_epipolar_image_gray.tif")
    img_right_path = os.path.join(gt_path, "right_epipolar_image_gray.tif")
    
    img_left = np.moveaxis(rasterio.open(os.path.join(gt_path, "left_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_left_gray = rasterio.open(img_left_path).read(1)
    img_right = np.moveaxis(rasterio.open(os.path.join(gt_path, "right_epipolar_image.tif")).read([1,2,3]), 0, -1)
    img_right_gray = rasterio.open(img_right_path).read(1)

    true_disp = rasterio.open(os.path.join(gt_path, "left_epipolar_disp.tif")).read(1)
    true_disp_mask = rasterio.open(os.path.join(gt_path, "left_epipolar_mask.tif")).read(1)
    true_disp[true_disp_mask > 0] = np.nan
    
    d_min, d_max = int(np.floor(np.nanmin(true_disp))), int(np.ceil(np.nanmax(true_disp)))
else:
    print("Wrong year!")
    raise KeyError
    
with open(os.path.join(output_path, "cfg/config.json")) as f:
    cfg = json.load(f)
    d_min, d_max = int(cfg["input"]["disp_min"]), int(cfg["input"]["disp_max"])
    win_size = int(cfg["pipeline"]["matching_cost"]["window_size"])

logging.info(f" {scene}\nd_min: {d_min} | d_max: {d_max}\nwin_size: {win_size}")

with warnings.catch_warnings():
    warnings.filterwarnings("ignore", category=rasterio.errors.NotGeoreferencedWarning)
    left_conf = rasterio.open(os.path.join(output_path, "left_confidence_measure.tif"))
    disp_map = rasterio.open(os.path.join(output_path, "left_disparity.tif")).read(1)
    validity_map = rasterio.open(os.path.join(output_path, "left_validity_mask.tif")).read(1).astype(dtype=np.uint16)
    PANDORA_MSK_PIXEL_INVALID  = (validity_map & 0b01111000011) > 0

validity_bitmap = np.flip(np.unpackbits(validity_map.view(dtype=np.uint8).reshape(*validity_map.shape, 2)[:,:,[1,0]]).reshape(*validity_map.shape, -1)[:,:,:], axis=2)[:,:,[2,8,11]].astype(bool)


conf_inf = left_conf.read(int(np.argwhere(np.array(left_conf.descriptions)=='confidence_from_interval_bounds_inf.interval_reg_90')[0, 0] + 1))
conf_sup = left_conf.read(int(np.argwhere(np.array(left_conf.descriptions)=='confidence_from_interval_bounds_sup.interval_reg_90')[0, 0] + 1))
conf_inf_no_reg = left_conf.read(int(np.argwhere(np.array(left_conf.descriptions)=='confidence_from_interval_bounds_inf.interval')[0, 0] + 1))
conf_sup_no_reg = left_conf.read(int(np.argwhere(np.array(left_conf.descriptions)=='confidence_from_interval_bounds_sup.interval')[0, 0] + 1))
conf_amb = left_conf.read(int(np.argwhere(np.array(left_conf.descriptions)=='confidence_from_ambiguity')[0, 0] + 1))


no_data = np.isnan(true_disp)
wrong_interval = (conf_inf > true_disp) | (conf_sup < true_disp)

### Figure 1

In [ ]:
row = 110

# Tableau colour bind palette [#006BA4, #FF800E, #ABABAB, #595959, #5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF]
blue = tuple(int("#006BA4".lstrip("#")[i:i+2], 16) for i in (0, 2, 4))
orange = tuple(int("#FF800E".lstrip("#")[i:i+2], 16) for i in (0, 2, 4))

### Stats for thetile
error_tot = np.nansum(wrong_interval[~no_data])/wrong_interval[~no_data].size*100
error_row = np.nansum(wrong_interval[row, :][~no_data[row, :]])/wrong_interval[row, :][~no_data[row, :]].size*100

### Creating the top left image 
image_error = np.full((*wrong_interval.shape, 4), 0, dtype=np.uint8)
image_error[wrong_interval, :] = orange + (255,)
image_error[no_data, :] = 0

blue = tuple(np.array(blue)/255)
orange = tuple(np.array(orange)/255)

fig = plt.figure(figsize=(5, 5))

# Ax
ax = fig.add_subplot(111)

ax.imshow(img_left_gray, vmin=np.nanquantile(img_left_gray, 0.01), vmax=np.nanquantile(img_left_gray, 0.99), cmap="gray")
ax.imshow(image_error)

ax.set_xlim([np.abs(d_min), img_left_gray.shape[1]])
#ax.set_title("Wrong interval (reg after)")
legend_elements = [
    Line2D([0], [0], marker="s", color=orange, label="Wrong interval", linewidth=0, markerfacecolor=orange, markersize=10, markeredgecolor="k", markeredgewidth=1),
    ]
#Line2D([0], [0], marker="s", color="w", label="Validity mask", markerfacecolor="pink", markersize=15, markeredgecolor="k", markeredgewidth=1)
ax.legend(handles=legend_elements, loc="upper left", frameon=True, framealpha=0.5, facecolor="white", fontsize=12)
ax.grid(color="gray", linewidth=1)

ax.axhline(row-1, c='white', linestyle="--", linewidth=4)
ax.axhline(row+1, c='white', linestyle="--", linewidth=4)
ax.axhline(row, c='k', linestyle="--", linewidth=4)
ax.tick_params(axis="y",  which='both', left=False, right=False, labelleft=False, reset=True)

# 2014
#ax.plot([500, 1500], [row - 1, row - 1], c='w', linestyle="--")
#ax.plot([500, 1500], [row + 1, row + 1], c='w', linestyle="--")
#ax.plot([500, 1500], [row, row], c='k', linestyle="--")


ax.grid(False)
plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_1_a.png", dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_1_a.png")

In [ ]:
# teddy
row = 110

fig = plt.figure(figsize=(4.2, 3))
# Ax
ax = fig.add_subplot(111)

### Plotting the predicted disparity, gt, intervals, for the selected row
# Removing incomplete disparity mask

disp_row = disp_map[row, :370].copy()
disp_row[disp_row > conf_sup[row, :370]] = conf_sup[row, :370][disp_row > conf_sup[row, :370]]
disp_row[disp_row < conf_inf[row, :370]] = conf_inf[row, :370][disp_row < conf_inf[row, :370]]

ax.plot(true_disp[row, :370], color=blue, label="True disparity", linewidth=2.5)
ax.plot(disp_row, color=orange, label="Computed disparity", linewidth=1)
ax.plot(conf_inf[row, :370], linestyle="--", color=orange, label="Confidence intervals", linewidth=1.5)
ax.plot(conf_sup[row, :370], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("Disparity", fontsize=16)
ax.tick_params(axis='both', labelsize=16)
ax.set_xlabel("Columns", fontsize=16)
#ax.set_title(f"{cost_function} - {scene} | Row {row}")

ax.legend(frameon=True, framealpha=1, facecolor="white")

# Grey area for invalid pixels

bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :370, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
#ax.add_collection(pc)

ax.set_xlim([np.abs(d_min), 370])
ax.set_ylim([-55, -10])
ax.grid(True)

#ax.legend(loc='upper left', frameon=True, framealpha=1, facecolor="white");
ax.legend(loc=(0.05,0.1), frameon=True, framealpha=0.5, facecolor="white");

#ax.legend(loc=(0.6, 0.7), frameon=True, framealpha=1, facecolor="white");

# 2005
#ax.set_ylim([-65, -45])
#ax.set_yticks([k for k in ax.get_yticks() if int(k)==k])
# 2014
#ax.set_xlim([600, 2650])
#ax.legend(loc=(0.0, 0.0), frameon=True, framealpha=1, facecolor="white");
# 2021
#ax.set_xlim([1415, 1890])
##ax.set_ylim([-210, -50])
#ax.set_ylim([-110, -70])
#ax.legend(loc=(0.2,0.05), frameon=True, framealpha=1, facecolor="white");

plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_1_b.png", dpi=300, bbox_inches="tight");
img = np.array(Image.open("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_1_b.png"))
img_sum = img.sum(axis=2)
for k in range(img.shape[0]):
    if (img_sum[k, :] != (255*4)).any():
        top = k
        break
for k in range(img.shape[0]):
    if (img_sum[img.shape[0] - 1 - k, :] != (255*4)).any():
        bottom = img.shape[0] - 1 - k
        break
for k in range(img.shape[1]):
    if (img_sum[:, k] != (255*4)).any():
        left = k
        break
for k in range(img.shape[1]):
    if (img_sum[:, img.shape[1] - 1 - k] != (255*4)).any():
        right = img.shape[1] - 1 - k
        break
img = img[top:bottom+1, :right+1]
Image.fromarray(img).save("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_1_b.png")

### Figure 2

In [ ]:
fig = plt.figure(figsize=(6,2))
ax = fig.add_subplot(111)
ax.plot(pandora_machine.left_cv.cost_volume.disp, pandora_machine.left_cv.cost_volume.sel({"row": 231, "col": 113}), linestyle=":", label="ambiguity: 0.20", linewidth=3)
ax.plot(pandora_machine.left_cv.cost_volume.disp, pandora_machine.left_cv.cost_volume.sel({"row": 207, "col": 419}), linestyle="--", label="ambiguity: 0.40", linewidth=3)
ax.plot(pandora_machine.left_cv.cost_volume.disp, pandora_machine.left_cv.cost_volume.sel({"row": 346, "col": 347}), c="k", label="ambiguity: 0.85", linewidth=3)
ax.legend()
ax.set_xlim([-60, 0])
ax.grid(True)
ax.tick_params(axis='both', labelsize=13)
ax.set_ylabel("Matching cost", fontsize=13)
ax.set_xlabel("Disparity", fontsize=13)
plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_2.png", dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_2.png")

In [ ]:
fig = plt.figure(figsize=(6,2))
ax = fig.add_subplot(111)

ax.axhline(0.9, linewidth=1, c="gray")


pi = (pandora_machine.left_cv.cost_volume.sel({"row":231 , "col": 113}) - max_) / (min_ - max_)
pi = pi + 1 - np.nanmax(pi)
ax.plot(pandora_machine.left_cv.cost_volume.disp, pi, linestyle=":", linewidth=3)
alpha = np.argwhere(pi.data>=0.9)
ax.plot([np.min(alpha)-60, np.min(alpha)-60], [0, 0.9], linestyle=":", c="#006BA4", linewidth=2)
ax.plot([np.max(alpha)-60, np.max(alpha)-60], [0, 0.9], linestyle=":", c="#006BA4", linewidth=2)


pi = (pandora_machine.left_cv.cost_volume.sel({"row": 207, "col": 419}) - max_) / (min_ - max_)
pi = pi + 1 - np.nanmax(pi)
ax.plot(pandora_machine.left_cv.cost_volume.disp, pi, linestyle="--", linewidth=3)
alpha = np.argwhere(pi.data>=0.9)
ax.plot([np.min(alpha)-60-0.7, np.min(alpha)-60-1], [0, 0.9], linestyle="--", c="#FF800E", linewidth=2)
ax.plot([np.max(alpha)-60, np.max(alpha)-60], [0, 0.9], linestyle="--", c="#FF800E", linewidth=2)

pi = (pandora_machine.left_cv.cost_volume.sel({"row": 346, "col": 347}) - max_) / (min_ - max_)
pi = pi + 1 - np.nanmax(pi)
ax.plot(pandora_machine.left_cv.cost_volume.disp, pi, c="k", linewidth=3)
alpha = np.argwhere(pi.data>=0.9)
ax.plot([np.min(alpha)-60, np.min(alpha)-60], [0, 0.92], c="k", linewidth=2)
ax.plot([np.max(alpha)-60, np.max(alpha)-60], [0, 0.91], c="k", linewidth=2)

ax.set_xlim([-60, 0])
ax.set_ylim([0, 1.05])

ax.grid(True)
ax.set_ylabel("Possibility", fontsize=13)
ax.tick_params(axis='both', labelsize=13)
ax.set_xlabel("Disparity", fontsize=13)

ax.set_yticks([0, 0.9, 1])
plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_3.png", dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_3.png")

### Figure 5

In [ ]:
### cones
row = 260

fig = plt.figure(figsize=(4.2, 3))
# Ax
ax = fig.add_subplot(111)

### Plotting the predicted disparity, gt, intervals, for the selected row
# Removing incomplete disparity mask

disp_row = disp_map[row, :370].copy()
disp_row[disp_row > conf_sup[row, :370]] = conf_sup[row, :370][disp_row > conf_sup[row, :370]]
disp_row[disp_row < conf_inf[row, :370]] = conf_inf[row, :370][disp_row < conf_inf[row, :370]]

ax.plot(true_disp[row, :370], color=blue, label="True disparity", linewidth=2.5)
ax.plot(disp_row, color=orange, label="Computed disparity", linewidth=1)
ax.plot(conf_inf_no_reg[row, :370], linestyle="--", color=orange, label="Confidence intervals", linewidth=1.5)
ax.plot(conf_sup_no_reg[row, :370], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("Disparity", fontsize=16)
ax.tick_params(axis='both', labelsize=16)
ax.set_xlabel("Columns", fontsize=16)
#ax.set_title(f"{cost_function} - {scene} | Row {row}")

ax.legend(frameon=True, framealpha=1, facecolor="white")

# Grey area for invalid pixels

bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :370, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)

ax.set_xlim([80, 250])
ax.set_ylim([-55, -25])
ax.grid(True)

#ax.legend(loc='upper left', frameon=True, framealpha=1, facecolor="white");
ax.legend(loc=(0.05,0.7), frameon=True, framealpha=0.5, facecolor="white");

#ax.legend(loc=(0.6, 0.7), frameon=True, framealpha=1, facecolor="white");

# 2005
#ax.set_ylim([-65, -45])
#ax.set_yticks([k for k in ax.get_yticks() if int(k)==k])
# 2014
#ax.set_xlim([600, 2650])
#ax.legend(loc=(0.0, 0.0), frameon=True, framealpha=1, facecolor="white");
# 2021
#ax.set_xlim([1415, 1890])
##ax.set_ylim([-210, -50])
#ax.set_ylim([-110, -70])
#ax.legend(loc=(0.2,0.05), frameon=True, framealpha=1, facecolor="white");

plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_5_a.png", dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_5_a.png")

In [ ]:
### cones
row = 260

fig = plt.figure(figsize=(4.2, 3))
# Ax
ax = fig.add_subplot(111)

### Plotting the predicted disparity, gt, intervals, for the selected row
# Removing incomplete disparity mask

disp_row = disp_map[row, :370].copy()
disp_row[disp_row > conf_sup[row, :370]] = conf_sup[row, :370][disp_row > conf_sup[row, :370]]
disp_row[disp_row < conf_inf[row, :370]] = conf_inf[row, :370][disp_row < conf_inf[row, :370]]

ax.plot(true_disp[row, :370], color=blue, label="True disparity", linewidth=2.5)
ax.plot(disp_row, color=orange, label="Computed disparity", linewidth=1)
ax.plot(conf_inf[row, :370], linestyle="--", color=orange, label="Confidence intervals", linewidth=1.5)
ax.plot(conf_sup[row, :370], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("Disparity", fontsize=16)
ax.tick_params(axis='both', labelsize=16)
ax.set_xlabel("Columns", fontsize=16)
#ax.set_title(f"{cost_function} - {scene} | Row {row}")

ax.legend(frameon=True, framealpha=1, facecolor="white")

# Grey area for invalid pixels

bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :370, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)

ax.set_xlim([80, 250])
ax.set_ylim([-55, -25])
ax.grid(True)

#ax.legend(loc='upper left', frameon=True, framealpha=1, facecolor="white");
ax.legend(loc=(0.05,0.7), frameon=True, framealpha=0.5, facecolor="white");

#ax.legend(loc=(0.6, 0.7), frameon=True, framealpha=1, facecolor="white");

# 2005
#ax.set_ylim([-65, -45])
#ax.set_yticks([k for k in ax.get_yticks() if int(k)==k])
# 2014
#ax.set_xlim([600, 2650])
#ax.legend(loc=(0.0, 0.0), frameon=True, framealpha=1, facecolor="white");
# 2021
#ax.set_xlim([1415, 1890])
##ax.set_ylim([-210, -50])
#ax.set_ylim([-110, -70])
#ax.legend(loc=(0.2,0.05), frameon=True, framealpha=1, facecolor="white");

plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_5_b.png", dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_5_b.png")

### Figure 6

In [ ]:
#cones
row = 230

fig = plt.figure(figsize=(5, 3))
# Ax
ax = fig.add_subplot(111)

### Plotting the predicted disparity, gt, intervals, for the selected row
# Removing incomplete disparity mask

disp_row = disp_map[row, :].copy()
disp_row[disp_row > conf_sup[row, :]] = conf_sup[row, :][disp_row > conf_sup[row, :]]
disp_row[disp_row < conf_inf[row, :]] = conf_inf[row, :][disp_row < conf_inf[row, :]]

ax.plot(disp_row, color=orange, label="Computed disparity", linewidth=1.5)
ax.plot(conf_inf[row, :], linestyle="--", color=orange, label="Confidence intervals", linewidth=1.5)
ax.plot(conf_sup[row, :], linestyle="--", color=orange, linewidth=1.5)
ax.plot(true_disp[row, :], color=blue, label="True disparity", linewidth=2.5)

ax.set_ylabel("Disparity")
ax.tick_params(axis='y')
ax.set_xlabel("Columns")
#ax.set_title(f"{cost_function} - {scene} | Row {row}")

ax.legend(frameon=True, framealpha=1, facecolor="white")

# Grey area for invalid pixels
ax.set_ylim([-60, 15])

bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)

ax.set_xlim([np.abs(d_min), disp_map.shape[1]])

ax.grid(False)

#ax.legend(loc='upper left', frameon=True, framealpha=1, facecolor="white");
ax.legend(loc=(0.495,0.71), frameon=True, framealpha=0.5, facecolor="white");

# 2003
ax.set_xlim([95, 132])
delta_max = np.nanmax(np.hstack([disp_row[100:127], true_disp[row, 100:126]]))
delta_min = np.nanmin(np.hstack([disp_row[100:127], true_disp[row, 100:126]]))
ax.plot([100, 126], [delta_max, delta_max], c="k", linestyle=":", linewidth=1.5)
ax.plot([100, 126], [delta_min, delta_min], c="k", linestyle=":", linewidth=1.5)


#ax.legend(loc=(0.6, 0.7), frameon=True, framealpha=1, facecolor="white");

# 2005
#ax.set_ylim([-65, -45])
#ax.set_yticks([k for k in ax.get_yticks() if int(k)==k])
# 2014
#ax.set_xlim([600, 2650])
#ax.legend(loc=(0.0, 0.0), frameon=True, framealpha=1, facecolor="white");
# 2021
#ax.set_xlim([1415, 1890])
##ax.set_ylim([-210, -50])
#ax.set_ylim([-110, -70])
#ax.legend(loc=(0.2,0.05), frameon=True, framealpha=1, facecolor="white");

plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/error_relative_%s_%s.png" % (scene, cost_function), dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_5_b.png")

### Figure 7

In [ ]:
### cones
row = 230

# Tableau colour bind palette [#006BA4, #FF800E, #ABABAB, #595959, #5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF]
blue = tuple(int("#006BA4".lstrip("#")[i:i+2], 16) for i in (0, 2, 4))
orange = tuple(int("#FF800E".lstrip("#")[i:i+2], 16) for i in (0, 2, 4))

### Stats for thetile
error_tot = np.nansum(wrong_interval[~no_data])/wrong_interval[~no_data].size*100
error_row = np.nansum(wrong_interval[row, :][~no_data[row, :]])/wrong_interval[row, :][~no_data[row, :]].size*100

### Creating the top left image 
image_error = np.full((*wrong_interval.shape, 4), 0, dtype=np.uint8)
image_error[wrong_interval, :] = orange + (255,)
image_error[no_data, :] = 0

blue = tuple(np.array(blue)/255)
orange = tuple(np.array(orange)/255)

fig = plt.figure(figsize=(5, 5))

# Ax
ax = fig.add_subplot(111)

ax.imshow(img_left_gray, vmin=np.nanquantile(img_left_gray, 0.01), vmax=np.nanquantile(img_left_gray, 0.99), cmap="gray")
ax.imshow(image_error)

ax.set_xlim([np.abs(d_min), img_left_gray.shape[1]])
#ax.set_title("Wrong interval (reg after)")
legend_elements = [
    Line2D([0], [0], marker="s", color=orange, label="Wrong interval", linewidth=0, markerfacecolor=orange, markersize=10, markeredgecolor="k", markeredgewidth=1),
    ]
#Line2D([0], [0], marker="s", color="w", label="Validity mask", markerfacecolor="pink", markersize=15, markeredgecolor="k", markeredgewidth=1)
ax.legend(handles=legend_elements, loc="upper left", frameon=True, framealpha=0.5, facecolor="white", fontsize=12)
ax.grid(color="gray", linewidth=1)

ax.axhline(row-1, c='white', linestyle="--", linewidth=4)
ax.axhline(row+1, c='white', linestyle="--", linewidth=4)
ax.axhline(row, c='k', linestyle="--", linewidth=4)
ax.tick_params(axis="y",  which='both', left=False, right=False, labelleft=False, reset=True)

# 2014
#ax.plot([500, 1500], [row - 1, row - 1], c='w', linestyle="--")
#ax.plot([500, 1500], [row + 1, row + 1], c='w', linestyle="--")
#ax.plot([500, 1500], [row, row], c='k', linestyle="--")


ax.grid(False)
plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_7_a.png", dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_7_a.png")

In [ ]:
fig = plt.figure(figsize=(4.2, 3))
# Ax
ax = fig.add_subplot(111)

### Plotting the predicted disparity, gt, intervals, for the selected row
# Removing incomplete disparity mask

disp_row = disp_map[row,:].copy()
disp_row[disp_row > conf_sup[row, :]] = conf_sup[row, :][disp_row > conf_sup[row, :]]
disp_row[disp_row < conf_inf[row, :]] = conf_inf[row, :][disp_row < conf_inf[row, :]]

ax.plot(true_disp[row, :], color=blue, label="True disparity", linewidth=2.5)
ax.plot(disp_row, color=orange, label="Computed disparity", linewidth=1)
ax.plot(conf_inf[row, :], linestyle="--", color=orange, label="Confidence intervals", linewidth=1.5)
ax.plot(conf_sup[row, :], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("Disparity", fontsize=16)
ax.tick_params(axis='both', labelsize=16)
ax.set_xlabel("Columns", fontsize=16)
#ax.set_title(f"{cost_function} - {scene} | Row {row}")

ax.legend(frameon=True, framealpha=1, facecolor="white")

# Grey area for invalid pixels

bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)

ax.set_xlim([90, 180])
#ax.set_ylim([-55, -25])
ax.grid(True)

#ax.legend(loc='upper left', frameon=True, framealpha=1, facecolor="white");
ax.legend(loc=(0.39,0.7), frameon=True, framealpha=0.5, facecolor="white");

#ax.legend(loc=(0.6, 0.7), frameon=True, framealpha=1, facecolor="white");

# 2005
#ax.set_ylim([-65, -45])
#ax.set_yticks([k for k in ax.get_yticks() if int(k)==k])
# 2014
#ax.set_xlim([600, 2650])
#ax.legend(loc=(0.0, 0.0), frameon=True, framealpha=1, facecolor="white");
# 2021
#ax.set_xlim([1415, 1890])
##ax.set_ylim([-210, -50])
#ax.set_ylim([-110, -70])
#ax.legend(loc=(0.2,0.05), frameon=True, framealpha=1, facecolor="white");

plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_7_b.png", dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_7_b.png")

In [ ]:
fig = plt.figure(figsize=(8.4, 3))
# Ax
ax = fig.add_subplot(111)

### Plotting the predicted disparity, gt, intervals, for the selected row
# Removing incomplete disparity mask

disp_row = disp_map[row,:].copy()
disp_row[disp_row > conf_sup[row, :]] = conf_sup[row, :][disp_row > conf_sup[row, :]]
disp_row[disp_row < conf_inf[row, :]] = conf_inf[row, :][disp_row < conf_inf[row, :]]

ax.plot(true_disp[row, :], color=blue, label="True disparity", linewidth=2.5)
ax.plot(disp_row, color=orange, label="Computed disparity", linewidth=1)
ax.plot(conf_inf[row, :], linestyle="--", color=orange, label="Confidence intervals", linewidth=1.5)
ax.plot(conf_sup[row, :], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("Disparity", fontsize=16)
ax.tick_params(axis='both', labelsize=16)
ax.set_xlabel("Columns", fontsize=16)
#ax.set_title(f"{cost_function} - {scene} | Row {row}")

ax.legend(frameon=True, framealpha=1, facecolor="white")

# Grey area for invalid pixels

bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)

ax.set_xlim([280, 448])
ax.set_ylim([-50, -20])
ax.grid(True)

#ax.legend(loc='upper left', frameon=True, framealpha=1, facecolor="white");
ax.legend(loc=(0.2,0.65), frameon=True, framealpha=0.5, facecolor="white");

#ax.legend(loc=(0.6, 0.7), frameon=True, framealpha=1, facecolor="white");

# 2005
#ax.set_ylim([-65, -45])
#ax.set_yticks([k for k in ax.get_yticks() if int(k)==k])
# 2014
#ax.set_xlim([600, 2650])
#ax.legend(loc=(0.0, 0.0), frameon=True, framealpha=1, facecolor="white");
# 2021
#ax.set_xlim([1415, 1890])
##ax.set_ylim([-210, -50])
#ax.set_ylim([-110, -70])
#ax.legend(loc=(0.2,0.05), frameon=True, framealpha=1, facecolor="white");

plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_7_c.png", dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_7_c.png")

### Figure 8

In [ ]:
### Jadeplant
row = 500

# Tableau colour bind palette [#006BA4, #FF800E, #ABABAB, #595959, #5F9ED1, #C85200, #898989, #A2C8EC, #FFBC79, #CFCFCF]
blue = tuple(int("#006BA4".lstrip("#")[i:i+2], 16) for i in (0, 2, 4))
orange = tuple(int("#FF800E".lstrip("#")[i:i+2], 16) for i in (0, 2, 4))

### Stats for thetile
error_tot = np.nansum(wrong_interval[~no_data])/wrong_interval[~no_data].size*100
error_row = np.nansum(wrong_interval[row, :][~no_data[row, :]])/wrong_interval[row, :][~no_data[row, :]].size*100

### Creating the top left image 
image_error = np.full((*wrong_interval.shape, 4), 0, dtype=np.uint8)
image_error[wrong_interval, :] = orange + (255,)
image_error[no_data, :] = 0

blue = tuple(np.array(blue)/255)
orange = tuple(np.array(orange)/255)

fig = plt.figure(figsize=(5, 5))

# Ax
ax = fig.add_subplot(111)

ax.imshow(img_left_gray, vmin=np.nanquantile(img_left_gray, 0.01), vmax=np.nanquantile(img_left_gray, 0.99), cmap="gray")
ax.imshow(image_error)

ax.set_xlim([np.abs(d_min), img_left_gray.shape[1]])
#ax.set_title("Wrong interval (reg after)")
legend_elements = [
    Line2D([0], [0], marker="s", color=orange, label="Wrong interval", linewidth=0, markerfacecolor=orange, markersize=10, markeredgecolor="k", markeredgewidth=1),
    ]
#Line2D([0], [0], marker="s", color="w", label="Validity mask", markerfacecolor="pink", markersize=15, markeredgecolor="k", markeredgewidth=1)
ax.legend(handles=legend_elements, loc="lower right", frameon=True, framealpha=0.5, facecolor="white", fontsize=12)
ax.grid(color="gray", linewidth=1)

ax.axhline(row-5, c='white', linestyle="--", linewidth=4)
ax.axhline(row+5, c='white', linestyle="--", linewidth=4)
ax.axhline(row, c='k', linestyle="--", linewidth=4)
ax.tick_params(axis="y",  which='both', left=False, right=False, labelleft=False, reset=True)

# 2014
#ax.plot([500, 1500], [row - 1, row - 1], c='w', linestyle="--")
#ax.plot([500, 1500], [row + 1, row + 1], c='w', linestyle="--")
#ax.plot([500, 1500], [row, row], c='k', linestyle="--")


ax.grid(False)
plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_8_a.png", dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_8_a.png")

In [ ]:
fig = plt.figure(figsize=(8.4, 3))
# Ax
ax = fig.add_subplot(111)

### Plotting the predicted disparity, gt, intervals, for the selected row
# Removing incomplete disparity mask

disp_row = disp_map[row,:].copy()
disp_row[disp_row > conf_sup[row, :]] = conf_sup[row, :][disp_row > conf_sup[row, :]]
disp_row[disp_row < conf_inf[row, :]] = conf_inf[row, :][disp_row < conf_inf[row, :]]

ax.plot(true_disp[row, :], color=blue, label="True disparity", linewidth=2.5)
ax.plot(disp_row, color=orange, label="Computed disparity", linewidth=1)
ax.plot(conf_inf[row, :], linestyle="--", color=orange, label="Confidence intervals", linewidth=1.5)
ax.plot(conf_sup[row, :], linestyle="--", color=orange, linewidth=1.5)

ax.set_ylabel("Disparity", fontsize=16)
ax.tick_params(axis='both', labelsize=16)
ax.set_xlabel("Columns", fontsize=16)
#ax.set_title(f"{cost_function} - {scene} | Row {row}")

ax.legend(frameon=True, framealpha=1, facecolor="white")

# Grey area for invalid pixels

bottom_, top_ = ax.get_ylim()
diffs = np.diff(np.hstack([[False], validity_bitmap[row, :, 2], [False]]).astype(int))
lefts_, rights_ = np.argwhere(diffs==1).flatten(), np.argwhere(diffs==-1).flatten() - 1
invalid_zones = [Rectangle((l_, bottom_), r_ - l_, top_ - bottom_) for l_, r_ in zip(lefts_, rights_)]
pc = PatchCollection(invalid_zones, facecolor="gray", alpha=0.3)
ax.add_collection(pc)

ax.set_xlim([920, 1180])
#ax.set_ylim([-50, -20])
ax.grid(True)

#ax.legend(loc='upper left', frameon=True, framealpha=1, facecolor="white");
ax.legend(loc=(0.7, 0.7), frameon=True, framealpha=0.7, facecolor="white");

#ax.legend(loc=(0.6, 0.7), frameon=True, framealpha=1, facecolor="white");

# 2005
#ax.set_ylim([-65, -45])
#ax.set_yticks([k for k in ax.get_yticks() if int(k)==k])
# 2014
#ax.set_xlim([600, 2650])
#ax.legend(loc=(0.0, 0.0), frameon=True, framealpha=1, facecolor="white");
# 2021
#ax.set_xlim([1415, 1890])
##ax.set_ylim([-210, -50])
#ax.set_ylim([-110, -70])
#ax.legend(loc=(0.2,0.05), frameon=True, framealpha=1, facecolor="white");

plt.savefig("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_8_b.png", dpi=300, bbox_inches="tight");
crop_img("/work/CAMPUS/users/malinoro/outputs/CVPR_tests/pyplot/figure_8_b.png")

### Other

In [ ]:
print(np.unique(df.columns.get_level_values("Cost Function")))
print(np.unique(df.columns.get_level_values("Masks")))
print(np.unique(df.columns.get_level_values("Method")))
print(np.unique(df.columns.get_level_values("Type")))

In [ ]:
df_copy = df.copy()
df_copy[("CENSUS", "Confident_incomplete", "reg_90", "Proportion")] = df_copy[("CENSUS", "Confident_incomplete", "reg_90", "N pixel")] / df_copy[("CENSUS", "Incomplete disp", "reg_90", "N pixel")]
df_copy[("MCCNN", "Confident_incomplete", "reg_90", "Proportion")] = df_copy[("MCCNN", "Confident_incomplete", "reg_90", "N pixel")] / df_copy[("MCCNN", "Incomplete disp", "reg_90", "N pixel")]
df_copy = df_copy[df_copy.columns.sortlevel()[0]]
df_copy = df_copy[[k for k in df_copy.columns if k[1:4] == ("Incomplete disp", "reg_90", "Error") or (k[1:4] in [("Confident_incomplete", "reg_90", j) for j in ['Q1', 'Q3', 'median', "Proportion"]])]]
df_copy = np.round(df_copy*100, 0).astype(np.uint8)

latex_data = np.array(df_copy)

In [ ]:
def df_to_latex(latex_data):
    latex_str = "\hline\n"
    for row in latex_data:
        for col in row:
            latex_str += f"${col}$ & "
        latex_str = latex_str[:-2] + "\\\ \n"
    latex_str+= "\hline"
    return latex_str

In [ ]:
df_CENSUS = pd.read_csv("/home/mp/malinoro/CAMPUS/outputs/CVPR_tests/stats_global_CENSUS.csv", header=[0,1], index_col=[0])
df_MCCNN = pd.read_csv("/home/mp/malinoro/CAMPUS/outputs/CVPR_tests/stats_global_MCCNN.csv", header=[0,1], index_col=[0])
for m, c in df_CENSUS.columns:
    if c=="relative_error":
        print(m, c)
        df_CENSUS[(m, c)] = 1 - df_CENSUS[(m, c)]
        df_MCCNN[(m, c)] = 1 - df_MCCNN[(m, c)]        

In [ ]:
df_CENSUS.columns

In [ ]:
columns = [('Incomplete disp', 'accuracy'), ('Incomplete disp', 'median'), ('Confident_incomplete', 'accuracy'), ('Confident_incomplete', 'median'),
           ('Not_confident_incomplete', 'accuracy'), ('Not_confident_incomplete', 'median'), ('Not_confident_incomplete', 'relative_error')]

latex_str = "\hline\hline\n"
flag = True
for row in df_CENSUS.index:
    if flag:
        color = "\cellcolor{gray!30}"
    else:
        color = "\cellcolor{lightgray!10}"
    latex_str += f"{color}${row}$ & "
    for col in columns:
        prefix_1 = prefix_2 = suffix_1 = suffix_2 = ""
        if col[1] == "accuracy":
            if np.round(df_CENSUS.loc[row, col], 3) >= np.round(df_MCCNN.loc[row, col], 3):
                prefix_1, suffix_1 = "\\textbf{", "}"
            if np.round(df_CENSUS.loc[row, col], 3) <= np.round(df_MCCNN.loc[row, col], 3):
                prefix_2, suffix_2 = "\\textbf{", "}"
        else:
            if np.round(df_CENSUS.loc[row, col], 3) <= np.round(df_MCCNN.loc[row, col], 3):
                prefix_1, suffix_1 = "\\textbf{", "}"
            if np.round(df_CENSUS.loc[row, col], 3) >= np.round(df_MCCNN.loc[row, col], 3):
                prefix_2, suffix_2 = "\\textbf{", "}"
        latex_str += f"{color}${prefix_1}{np.round(df_CENSUS.loc[row, col], 3)}{suffix_1}$ & {color}${prefix_2}{np.round(df_MCCNN.loc[row, col], 3)}{suffix_2}$ & "
    latex_str = latex_str[:-2] + "\\\ \n"
    flag = not flag
latex_str += "\hline"
print(latex_str)

In [ ]:
interval_inf = np.arange(1, 7 * 10 + 1, dtype=np.float32).reshape((7, 10))
interval_sup = np.arange(5, 7 * 10 + 5, dtype=np.float32).reshape((7, 10))

gt_left = np.array(
    [
        [1.0, 3.4, 3.4, 3.4, 3.4, 6.0, 3.4, 3.4, 3.4, 3.4],
        [11.0, 12.0, 13.0, 3.4, 3.4, 3.4, 3.4, 18.0, 19.0, 20.0],
        [21.0, 3.4, 3.4, 3.4, 25.0, 26.0, 27.0, 28.0, 29.0, 30.0],
        [31.0, 32.0, 33.0, 34.0, 35.1, 35.1, 37.0, 38.7, 38.7, 38.7],
        [41.0, 42.0, 43.0, 44.0, 45.0, 46.0, 47.0, 48.0, 38.9, 38.9],
        [51.0, 52.0, 53.1, 53.1, 55.0, 56.0, 57.0, 38.9, 38.9, 38.9],
        [61.0, 62.0, 63.0, 64.0, 65.0, 66.0, 67.0, 68.0, 49.6, 49.6],
    ],
    dtype=np.float32,
)

gt_right = np.array(
    [
        [5.0, 26.6, 26.6, 26.6, 26.6, 10.0, 26.6, 26.6, 26.6, 26.6],
        [15.0, 16.0, 17.0, 26.6, 26.6, 26.6, 26.6, 22.0, 23.0, 24.0],
        [25.0, 26.6, 26.6, 26.6, 29.0, 30.0, 31.0, 32.0, 33.0, 34.0],
        [35.0, 36.0, 37.0, 38.0, 39.9, 39.9, 41.0, 63.3, 63.3, 63.3],
        [45.0, 46.0, 47.0, 48.0, 49.0, 50.0, 51.0, 52.0, 73.1, 73.1],
        [55.0, 56.0, 57.9, 57.9, 59.0, 60.0, 61.0, 73.1, 73.1, 73.1],
        [65.0, 66.0, 67.0, 68.0, 69.0, 70.0, 71.0, 72.0, 73.4, 73.4],
    ],
    dtype=np.float32,
)

gt_connection_matrix = np.array(
    [
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [1, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 1, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 0, 1, 1, 0, 1, 1],
        [0, 0, 0, 0, 0, 0, 1, 0, 1, 1],
    ],
    dtype=bool,
)

gt_mask_modif = np.array(
    [
        [0, 1, 1, 1, 1, 0, 1, 1, 1, 0],
        [0, 0, 0, 1, 1, 1, 1, 0, 0, 0],
        [0, 1, 1, 1, 0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 1, 1, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 0, 0, 0, 1, 1, 0],
        [0, 0, 0, 0, 0, 0, 0, 0, 1, 0],
    ],
    dtype=np.bool_,
)

border_left = np.array([[0, 1], [0, 6], [1, 3], [2, 1], [3, 4], [3, 7], [4, 8], [5, 2], [5, 7], [6, 8]])
border_right = np.array([[0, 4], [0, 8], [1, 6], [2, 3], [3, 5], [3, 8], [4, 8], [5, 3], [5, 8], [6, 8]])

In [ ]:
reg_left, reg_right, graph = pandora.interval_tools.graph_regularization(
    interval_inf, interval_sup, border_left, border_right, gt_connection_matrix, quantile=0.9
)

In [ ]:
reg_left, reg_right, graph.astype(int)

In [ ]:
gt_left, gt_right, gt_mask_modif.astype(int)

In [ ]:
ambiguity = np.array(
    [
        [1.0, 0.2, 0.2, 0.2, 0.2, 1.0, 0.2, 0.2, 0.2, 0.2],
        [1.0, 1.0, 1.0, 0.2, 0.2, 0.2, 0.2, 1.0, 1.0, 1.0],
        [1.0, 0.2, 0.2, 0.2, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0],
        [1.0, 1.0, 1.0, 1.0, 0.2, 0.2, 1.0, 0.2, 0.2, 0.2],
        [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 0.2],
        [1.0, 1.0, 0.2, 0.2, 1.0, 1.0, 1.0, 0.2, 0.2, 0.2],
        [1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.2, 0.2],
    ],
    dtype=np.float32,
)
ambiguity <= 0.6

n_row, _ = ambiguity.shape
pad = 1 // 2
minimized_conf_from_amb = np.hstack((np.ones((n_row, pad)), ambiguity, np.ones((n_row, pad))))
# H-stacking is to conserve the same shape after view.
# Because we take the minimum afterwards, it needs to be ones
minimized_conf_from_amb = np.nanmin(
    np.lib.stride_tricks.sliding_window_view(minimized_conf_from_amb, 1, axis=1),
    axis=-1,
)

# Final column to 1 and H-stacking is to unsure that
# we always start with a left border and end with a right border
minimized_conf_from_amb[:, -1] = 1
border = np.diff(
    np.hstack([np.ones((minimized_conf_from_amb.shape[0], 1)), minimized_conf_from_amb >= 0.6]),
    axis=-1,
)
border_left_ = np.argwhere(border == -1)
border_right_ = np.argwhere(border == 1)
border_right_[:, 1] = border_right_[:, 1] - 1

In [ ]:
np.hstack((border_left[:,1:], border_left_[:,1:], border_right[:,1:], border_right_[:,1:]))

In [ ]:
for k in border_left_:
    print(f"[{k[0]}, {k[1]}], ", end="")

In [ ]:
interval_inf = np.arange(1, 7 * 10 + 1, dtype=np.float32).reshape((7, 10))
interval_sup = np.arange(5, 7 * 10 + 5, dtype=np.float32).reshape((7, 10))
interval_inf, interval_sup

In [ ]:
matrix = np.zeros(minimized_conf_from_amb.shape, dtype=int)
for k in range(border_left.shape[0]):
    matrix[border_left[k,0], border_left[k,1]:border_right[k, 1]+1] = 1
matrix

In [ ]:
np.nanquantile(,0.9)

In [ ]:
pandora.interval_tools.graph_regularization(interval_inf, interval_sup, border_left, border_right, graph, 0.9)

In [ ]:
gt_left

In [ ]:
gt_left